In [1]:
import dspy
from dspy.datasets import HotPotQA

lm = dspy.LM('openai/gpt-4o-mini')
colbert = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.configure(lm=lm, rm=colbert)

agent = dspy.ReAct("question -> answer", tools=[dspy.Retrieve(k=1)])

/workspace/hello-dspy/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Disable litellm caching to avoid the error seen in the output
import os
import litellm
litellm.cache = None

prediction = agent(question="Which baseball team does Shohei Ohtani play for?")
print(prediction.answer)

Shohei Ohtani plays for the Los Angeles Angels.


In [6]:
dspy.inspect_history(n=5)





[2025-03-14T04:54:26.226297]

System message:

Your input fields are:
1. `question` (str)
2. `trajectory` (str)

Your output fields are:
1. `next_thought` (str)
2. `next_tool_name` (Literal['Retrieve', 'finish'])
3. `next_tool_args` (dict[str, Any])

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## trajectory ## ]]
{trajectory}

[[ ## next_thought ## ]]
{next_thought}

[[ ## next_tool_name ## ]]
{next_tool_name}        # note: the value you produce must exactly match (no extra characters) one of: Retrieve; finish

[[ ## next_tool_args ## ]]
{next_tool_args}        # note: the value you produce must adhere to the JSON schema: {"type": "object"}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `answer`.
        
        You will be given `question` and your goal is to finish with `answer`.
        
        To do this

In [15]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")


# This is optional. Create an MLflow Experiment to store and organize your traces.
mlflow.set_experiment("DSPy")


mlflow.dspy.autolog()

In [16]:
agent(question="Which baseball team does Shohei Ohtani play for?")

Prediction(
    trajectory={'thought_0': 'I need to find out which baseball team Shohei Ohtani currently plays for. This information is likely available in a sports database or recent news articles.', 'tool_name_0': 'Retrieve', 'tool_args_0': {'args': 'Shohei Ohtani current team', 'kwargs': {}}, 'observation_0': "Failed to execute: Retrieve.forward() missing 1 required positional argument: 'query'", 'thought_1': "I need to correct the arguments for the Retrieve tool to properly query for Shohei Ohtani's current team. I will ensure to provide the correct format for the query.", 'tool_name_1': 'Retrieve', 'tool_args_1': {'args': 'Shohei Ohtani current team', 'kwargs': {}}, 'observation_1': "Failed to execute: Retrieve.forward() missing 1 required positional argument: 'query'", 'thought_2': 'I need to adjust the arguments for the Retrieve tool to include the correct query format. I will specify the query as a dictionary to ensure it is executed properly.', 'tool_name_2': 'Retrieve', 'tool

Trace(request_id=1b896206f1004fc2bd9674bdcc89ec70)

In [20]:
from dspy.predict.react import Tool
from tavily import TavilyClient

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

search_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

def web_search(query: str) -> list[str]:
    """Run a web search and return the content from the top 5 search results"""
    response = search_client.search(query)
    return [r["content"] for r in response["results"]]

agent = dspy.ReAct("question -> answer", tools=[Tool(web_search)])

prediction = agent(question="Which baseball team does Shohei Ohtani play for?")
print(prediction.answer)

Shohei Ohtani currently plays for the Los Angeles Dodgers.


Trace(request_id=d87f39806e6f43e4a9b61022d9387779)

In [21]:
import dspy
from dspy.utils.callback import BaseCallback

# 1. Define a custom callback class that extends BaseCallback class
class AgentLoggingCallback(BaseCallback):

    # 2. Implement on_module_end handler to run a custom logging code.
    def on_module_end(self, call_id, outputs, exception):
        step = "Reasoning" if self._is_reasoning_output(outputs) else "Acting"
        print(f"== {step} Step ===")
        for k, v in outputs.items():
            print(f"  {k}: {v}")
        print("\n")

    def _is_reasoning_output(self, outputs):
        return any(k.startswith("Thought") for k in outputs.keys())

# 3. Set the callback to DSPy setting so it will be applied to program execution
dspy.configure(callbacks=[AgentLoggingCallback()])

In [ ]:
prediction = agent(question="Which baseball team does Shohei Ohtani play for?")
print(prediction.answer)

== Acting Step ===
  next_thought: I need to find out which baseball team Shohei Ohtani currently plays for, as this information may have changed recently.
  next_tool_name: web_search
  next_tool_args: {'query': 'Shohei Ohtani current team 2023'}


